In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image
import cv2
import pickle as pkl
# jupyter notebook magic to display plots in output
%matplotlib inline

In [ ]:
#mount drive to pull csv. Only works if you make a shortcut out of the submission folder
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Final_Project_Submissions_and_Files/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/.shortcut-targets-by-id/19rXw0lw__ZgpIRA9o5KsJsAmOyq3r8Pb/Final_Project_Submissions_and_Files


# Get Berkleley Coordinates

In [ ]:
coor = pd.read_csv('berkeley_latlon.csv')

In [ ]:
long = coor['longitude'].tolist()
lat = coor['latitude'].tolist()
str(long[0]) + ',' + str(lat[0])

'37.85399590164562,-122.24576845323176'

In [ ]:
long_lat_str = []
for i in range(len(long)):
  temp_str = str(long[i]) + ',' + str(lat[i])
  long_lat_str.append(temp_str)

In [ ]:
# len(long_lat_str)

6969

In [ ]:
final_coor_3000 = long_lat_str[:3000]
len(final_coor_3000)

3000

In [ ]:
split_coords = [re.split(r',',i) for i in final_coor_3000]
lat = []
lon = []
for pair in split_coords:
    temp_lat, temp_lon = pair
    lat.append(temp_lat)
    lon.append(temp_lon)

In [ ]:
# API key
meta_base = 'https://maps.googleapis.com/maps/api/streetview/metadata?'
pic_base = 'https://maps.googleapis.com/maps/api/streetview?'
api_key = 'your_api_key'

# This will be the input from anvil
location = '2121 I St NW, Washington, DC 20052'

# Set Up Google Static Street View API

In [ ]:
class StreetViewer(object):
    def __init__(self, api_key, location, size="640x640", pitch=-90,
                 folder_directory='/content/drive/MyDrive/imaing_data/', verbose=True):
        """
        This class handles a single API request to the Google Static Street View API
        api_key: obtain it from your Google Cloud Platform console
        location: the address string or a (lat, lng) tuple
        size: returned picture size. maximum is 640*640
        folder_directory: directory to save the returned objects from request
        verbose: whether to print the processing status of the request
        """
        # input params are saved as attributes for later reference
        self._key = api_key
        self.location = location
        self.size = size
        self.pitch = pitch
        self.folder_directory = folder_directory
        # call parames are saved as internal params
        self._meta_params = dict(key=self._key,
                                location=self.location)
        self._pic_params = dict(key=self._key,
                               location=self.location,
                               size=self.size,
                                pitch=self.pitch)
        self.verbose = verbose

    def get_meta(self):
        """
        Method to query the metadata of the address
        """
        # saving the metadata as json for later usage
        # "/"s are removed to avoid confusion on directory
        self.meta_path = "{}meta_{}.json".format(
            self.folder_directory, self.location.replace("/", ""))
        self._meta_response = requests.get(
            'https://maps.googleapis.com/maps/api/streetview/metadata?',
            params=self._meta_params)
        # turning the contents as meta_info attribute
        self.meta_info = self._meta_response.json()
        # meta_status attribute is used in get_pic method to avoid
        # query when no picture will be available
        self.meta_status = self.meta_info['status']
        if self._meta_response.ok:
            if self.verbose:
                print(">>> Obtained Meta from StreetView API:")
                print(self.meta_info)
            with open(self.meta_path, 'w') as file:
                json.dump(self.meta_info, file)
        else:
            print(">>> Failed to obtain Meta from StreetView API!!!")
        self._meta_response.close()

    def get_pic(self):
        """
        Method to query the StreetView picture and save to local directory
        """
        # define path to save picture and headers
        self.pic_path = "{}pic_{}.jpg".format(
            self.folder_directory, self.location.replace("/", ""))
        self.header_path = "{}header_{}.json".format(
            self.folder_directory, self.location.replace("/", ""))
        # only when meta_status is OK will the code run to query picture (cost incurred)
        if self.meta_status == 'OK':
            if self.verbose:
                print(">>> Picture available, requesting now...")
            self._pic_response = requests.get(
                'https://maps.googleapis.com/maps/api/streetview?',
                params=self._pic_params)
            self.pic_header = dict(self._pic_response.headers)
            if self._pic_response.ok:
                if self.verbose:
                    print(f">>> Saving objects to {self.folder_directory}")
                with open(self.pic_path, 'wb') as file:
                    file.write(self._pic_response.content)
                with open(self.header_path, 'w') as file:
                    json.dump(self.pic_header, file)
                self._pic_response.close()
                if self.verbose:
                    print(">>> COMPLETE!")
        else:
            print(">>> Picture not available in StreetView, ABORTING!")

    def display_pic(self):
        """
        Method to display the downloaded street view picture if available
        """
        if self.meta_status == 'OK':
            plt.figure(figsize=(10, 10))
            img=mpimg.imread(self.pic_path)
            imgplot = plt.imshow(img)
            plt.show()
        else:
            print(">>> Picture not available in StreetView, ABORTING!")

In [ ]:
%cd /content/drive/MyDrive/imaing_data/

/content/drive/.shortcut-targets-by-id/19rXw0lw__ZgpIRA9o5KsJsAmOyq3r8Pb/Final_Project_Submissions_and_Files/imaing_data


# Call API (commented out, don't call)

In [ ]:
img = []
new_long_lat_str = []

for i, street in enumerate(long_lat_str[:50]):

  # folder_directory='/content/drive/MyDrive/streetviews/'

  #Don't run
  # street_viewer_start = StreetViewer(api_key=api_key,location= street)
  # start = street_viewer_start.get_meta()
  # start_pic = street_viewer_start.get_pic()

  try:
    img_temp = image.load_img(street_viewer_start.pic_path)
    plt.imshow(img_temp)
    print(np.array(img_temp).shape)
    img_temp = np.array(img_temp)
    img_temp = cv2.resize(img_temp, (133,133))
    img.append(img_temp)
    new_long_lat_str.append(street)

  except FileNotFoundError:
    print('File is not valid for google steet api')

NameError: ignored

In [ ]:
for i, street in enumerate(long_lat_str[50:100]):

  # folder_directory='/content/drive/MyDrive/streetviews/'

  #Don't run
  # street_viewer_start = StreetViewer(api_key=api_key,location= street)
  # start = street_viewer_start.get_meta()
  # start_pic = street_viewer_start.get_pic()

  try:
    img_temp = image.load_img(street_viewer_start.pic_path)
    plt.imshow(img_temp)
    print(np.array(img_temp).shape)
    img_temp = np.array(img_temp)
    img_temp = cv2.resize(img_temp, (133,133))
    img.append(img_temp)
    new_long_lat_str.append(street)

  except FileNotFoundError:
    print('File is not valid for google steet api')

In [ ]:
for i, street in enumerate(long_lat_str[100:1000]):

  # folder_directory='/content/drive/MyDrive/streetviews/'

  #Don't run
  # street_viewer_start = StreetViewer(api_key=api_key,location= street)
  # start = street_viewer_start.get_meta()
  # start_pic = street_viewer_start.get_pic()

  try:
    img_temp = image.load_img(street_viewer_start.pic_path)
    plt.imshow(img_temp)
    print(np.array(img_temp).shape)
    img_temp = np.array(img_temp)
    img_temp = cv2.resize(img_temp, (133,133))
    img.append(img_temp)
    new_long_lat_str.append(street)

  except FileNotFoundError:
    print('File is not valid for google steet api')

# Construct dataframe for simulation
 - PCI
 - LAT, LON

In [ ]:
%cd /content/drive/MyDrive/streetviews2/arrs_pkl/

/content/drive/.shortcut-targets-by-id/1gnVV0eOEygqj9_wnneoTbAKcSf-W4nJS/streetviews2/arrs_pkl


In [ ]:
coor_store = []
sim = []

# with open('pci_3000_prediction_arr.csv','rb') as f:
#   berk_pci.append(pkl.load(f).tolist())


# with open('coord_first_1000_arr.csv','rb') as f:
#   coor_store.append(pkl.load(f).tolist())

with open('new_plot_arr_3000_arr2.csv','rb') as f:
  sim.append(pkl.load(f).tolist())

In [ ]:
split_coords = [re.split(r',',i) for i in coor_store[0]] #splits pair into individual
# split_coords

In [ ]:
lat = []
lon = []
for pair in split_coords:
    temp_lat, temp_lon = pair
    lat.append(temp_lat)
    lon.append(temp_lon)

In [ ]:
%cd /content/drive/MyDrive/Final_Project_Submissions_and_Files/

/content/drive/.shortcut-targets-by-id/19rXw0lw__ZgpIRA9o5KsJsAmOyq3r8Pb/Final_Project_Submissions_and_Files


In [ ]:
# Read PCI predictions
pre_PCI = []

with open('pci_3000_prediction_arr.csv','rb') as f:
  pre_PCI.append(pkl.load(f).tolist())

In [ ]:
final_df = pd.DataFrame(list(zip(lat,lon)),columns=['latitude','longitude'])
final_df['PCI'] = pre_PCI[0]
final_df['depreciation'] = sim[0]
final_df

,latitude,longitude,PCI,appreciation
0,37.85399590164562,-122.24576845323176,2,3.283824
1,37.853864060491375,-122.22240001625444,2,3.432602
2,37.8530456043654,-122.22293680704696,1,3.253338
3,37.854215682611695,-122.24441910539876,0,2.669549
4,37.85424202266612,-122.24328691985671,1,3.174045
...,...,...,...,...
2995,37.86428759277844,-122.2790953525607,2,3.228328
2996,37.84037863434792,-122.26360316532914,3,3.697436
2997,37.84143033234995,-122.27667312390132,3,3.733851
2998,37.838258084455774,-122.25801792752902,1,2.857390


In [ ]:
%cd /content/drive/MyDrive/Final_Project_Submissions_and_Files/

/content/drive/.shortcut-targets-by-id/19rXw0lw__ZgpIRA9o5KsJsAmOyq3r8Pb/Final_Project_Submissions_and_Files


# Input Dataframe for Carto input

In [ ]:
# Dont' run again
# final_df.to_csv('berkeley_pci_sim_3000.csv',index=False, encoding='utf-8')

#Simulating the Effect of Average Annual Temperature on Road Depreciation in Berkeley

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [ ]:
temp_data = pd.read_csv('city_temperature.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
temp_data = temp_data[temp_data['Country'] == 'US']
temp_data['Celsius'] = ((temp_data['AvgTemperature'] - 32) * (5/9)).round(2)
numberone = temp_data.groupby(['City', 'Year']).mean()['Celsius'].reset_index()
avg_temps = numberone.drop(['Year'], axis = 1).groupby(['City']).mean().reset_index().rename(columns = {'City': 'City', 'Celsius':'Avg Temp'})
stds = numberone.drop(['Year'], axis = 1).groupby(['City']).std().reset_index().rename(columns = {'City': 'City2', 'Celsius': 'STD'})
agg_df = pd.concat([avg_temps, stds], axis = 1).drop(['City2'], axis = 1)
agg_df = agg_df.set_index('City')
agg_df.rename(index = {'San Francisco': 'Berkeley'}, inplace = True)
agg_df = agg_df.reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def pa(predicted_PCI):
  if predicted_PCI >= 70:
    pa = 1/2 * (100.1 - predicted_PCI)
  elif predicted_PCI >= 55:
    pa = 1/3 * (100.1 - predicted_PCI)
  elif predicted_PCI >= 40:
    pa = 1/2.5 * (100.1 - predicted_PCI)
  elif predicted_PCI >= 25:
    pa = 1/2 * (100.1 - predicted_PCI)
  else:
    pa = 1/1.5 * (100.1 - predicted_PCI)
  return pa
def simulated_temperature_depreciation(predicted_PCI):
  if predicted_PCI == 0:
    PCI = np.random.uniform(85, 100)
  elif predicted_PCI == 1:
    PCI = np.random.uniform(70, 85)
  elif predicted_PCI == 2:
    PCI = np.random.uniform(55, 70)
  elif predicted_PCI == 3:
    PCI = np.random.uniform(40, 55)
  elif predicted_PCI == 4:
    PCI = np.random.uniform(25, 40)
  else:
    PCI = np.random.uniform(0, 25)
  arr = []
  for i in np.arange(1000):
    the_city = agg_df[agg_df['City'] == 'Berkeley']
    AAT = np.random.normal(the_city['Avg Temp'], the_city['STD'])[0]
    estimate = 130.36 - (4.56 * pa(PCI)) - (AAT * 3.98) + (.1988 * (AAT)**2)
    depreciation_est = -0.000121628 * (estimate**2) - .00523004 * estimate + 3.95978
    arr.append(depreciation_est)
  return np.mean(arr)

In [ ]:
plot_arr = []
for i in np.arange(len(pre[0])): # 6
  result = simulated_temperature_depreciation(pre[0][i])
  plot_arr.append(result)
#import matplotlib.pyplot as plt
# plt.plot(np.arange(101), plot_arr)

In [ ]:
plot_arr

In [ ]:
min(plot_arr)

0.5585396199960977

In [ ]:
len(plot_arr)

3000

# Final simulation csv (list) for Depreciation, dont' run

In [ ]:
# new_plot_arr= np.array(plot_arr)
# with open('new_plot_arr_3000_arr2.csv','wb') as f:
#   pkl.dump(new_plot_arr, f)